### TODO
* Collect exchange rates from bank of canada's api.
* Upload expenses which is in excel format.
* Merge the datasets and then clean and harmonize. 
* Upload final dataset to a MySQL database table.

In [1]:
import requests
import sys
import json
import datetime
import decimal
import petl
import pymysql 

In [2]:
startDate = '2020-01-01'
url = 'https://www.bankofcanada.ca/valet/observations/FXUSDCAD/json?start_date='

In [3]:
# request data from URL
try:
    BOCResponse = requests.get(url+startDate)
except Exception as e:
    print('could not make request:' + str(e))
    sys.exit()

In [4]:
# The HTTP 200 status response code indicates that the request has benn succeeded
print (BOCResponse)

<Response [200]>


In [5]:
# let's take a look at the data that's coming to us live from the API, it should be in JSON format
print (BOCResponse.text)

{
    "terms": {
        "url": "https://www.bankofcanada.ca/terms/"
    },
    "seriesDetail": {
        "FXUSDCAD": {
            "label": "USD/CAD",
            "description": "US dollar to Canadian dollar daily exchange rate",
            "dimension": {
                "key": "d",
                "name": "date"
            }
        }
    },
    "observations": [
        {
            "d": "2020-01-02",
            "FXUSDCAD": {
                "v": "1.2992"
            }
        },
        {
            "d": "2020-01-03",
            "FXUSDCAD": {
                "v": "1.2988"
            }
        },
        {
            "d": "2020-01-06",
            "FXUSDCAD": {
                "v": "1.2970"
            }
        },
        {
            "d": "2020-01-07",
            "FXUSDCAD": {
                "v": "1.3009"
            }
        },
        {
            "d": "2020-01-08",
            "FXUSDCAD": {
                "v": "1.3026"
            }
        },
        {
          

In [6]:
# initialize list of lists for data storage
BOCDates = []
BOCRates = []

In [7]:
# check response status and process BOC JSON object
if (BOCResponse.status_code == 200):
    BOCRaw = json.loads(BOCResponse.text)

In [8]:
# extract observation data into column arrays
for row in BOCRaw['observations']:
    BOCDates.append(datetime.datetime.strptime(row['d'],'%Y-%m-%d'))
    BOCRates.append(decimal.Decimal(row['FXUSDCAD']['v']))

In [9]:
BOCDates[:5]

[datetime.datetime(2020, 1, 2, 0, 0),
 datetime.datetime(2020, 1, 3, 0, 0),
 datetime.datetime(2020, 1, 6, 0, 0),
 datetime.datetime(2020, 1, 7, 0, 0),
 datetime.datetime(2020, 1, 8, 0, 0)]

In [10]:
BOCRates[:5]

[Decimal('1.2992'),
 Decimal('1.2988'),
 Decimal('1.2970'),
 Decimal('1.3009'),
 Decimal('1.3026')]

In [11]:
# create petl table from column arrays and rename the columns
exchangeRates = petl.fromcolumns([BOCDates,BOCRates],header=['date','rate'])

In [12]:
exchangeRates.head()

date,rate
2020-01-02 00:00:00,1.2992
2020-01-03 00:00:00,1.2988
2020-01-06 00:00:00,1.2970
2020-01-07 00:00:00,1.3009
2020-01-08 00:00:00,1.3026


In [13]:
# load expense document from local folder
try:
    expenses = petl.io.xlsx.fromxlsx('Expenses.xlsx',sheet='inUSD')
except Exception as e:
    print('could not open expenses.xlsx:' + str(e))
    sys.exit()

In [14]:
expenses.head()

date,USD
2020-01-11 00:00:00,25
2020-01-30 00:00:00,7
2020-02-11 00:00:00,25
2020-02-28 00:00:00,7
2020-03-11 00:00:00,25


In [15]:
# join tables with outer join to have every observation from both tables
expenses_ds = petl.outerjoin(exchangeRates,expenses,key='date')

In [16]:
expenses_ds.head()

date,rate,USD
2020-01-02 00:00:00,1.2992,None
2020-01-03 00:00:00,1.2988,None
2020-01-06 00:00:00,1.2970,None
2020-01-07 00:00:00,1.3009,None
2020-01-08 00:00:00,1.3026,None


In [17]:
# fill down missing values, since expenses are daily but there are not rates for weekends
expenses_ds = petl.filldown(expenses_ds,'rate')

In [18]:
# remove dates with no expenses, we don't need those observations in the final dataset
expenses_ds = petl.select(expenses_ds,lambda rec: rec.USD != None)

In [19]:
expenses_ds.head()

date,rate,USD
2020-01-11 00:00:00,1.3051,25
2020-01-30 00:00:00,1.3217,7
2020-02-11 00:00:00,1.3292,25
2020-02-28 00:00:00,1.3429,7
2020-03-11 00:00:00,1.3745,25


In [20]:
# add CDN column
expenses_ds = petl.addfield(expenses_ds,'CAD', lambda rec: decimal.Decimal(rec.USD) * rec.rate)

In [21]:
expenses_ds.head()

date,rate,USD,CAD
2020-01-11 00:00:00,1.3051,25,32.6275
2020-01-30 00:00:00,1.3217,7,9.2519
2020-02-11 00:00:00,1.3292,25,33.2300
2020-02-28 00:00:00,1.3429,7,9.4003
2020-03-11 00:00:00,1.3745,25,34.3625


In [22]:
# intialize database connection
try:
    dbConnection = pymysql.connect(host='localhost', user='root', password='12345678', db='etldemo')
except Exception as e:
    print('could not connect to database:' + str(e))
    sys.exit()

In [23]:
cur = dbConnection.cursor()
cur.execute('SET SQL_MODE=ANSI_QUOTES')

0

In [24]:
# populate Expenses database table
try:
    petl.io.todb (expenses_ds,dbConnection,'expenses')
except Exception as e:
    print('could not write to database:' + str(e))
print (expenses_ds)

+---------------------+--------+-----+---------+
| date                | rate   | USD | CAD     |
+=====================+========+=====+=========+
| 2020-01-11 00:00:00 | 1.3051 |  25 | 32.6275 |
+---------------------+--------+-----+---------+
| 2020-01-30 00:00:00 | 1.3217 |   7 | 9.2519  |
+---------------------+--------+-----+---------+
| 2020-02-11 00:00:00 | 1.3292 |  25 | 33.2300 |
+---------------------+--------+-----+---------+
| 2020-02-28 00:00:00 | 1.3429 |   7 | 9.4003  |
+---------------------+--------+-----+---------+
| 2020-03-11 00:00:00 | 1.3745 |  25 | 34.3625 |
+---------------------+--------+-----+---------+
...



P.S. This notebook has been created with few changes while following Hackforge tech talk with Doug Sartori demonstrating ETL with Python and PETL. Here's the [link](https://youtu.be/InLgSUw_ZOE) to the video.